In [3]:
import pandas as pd
import numpy as np
import spacy
import random
import es_core_news_sm

random.seed(0)

In [4]:
# data import
data = pd.read_excel(r'../data/Datos Codificados Estudio v2a.xlsx')

In [5]:
data_matrix = pd.concat(
    (data.iloc[:,2],    # Cue/Concept
    data.iloc[:,6],     # Codification
    data.iloc[:,3]      # Description of the cue
    ),
    axis = 1
)

In [56]:
data_matrix

,Palabra (concepto),Codificación,Descripción
0,Compasión,sentimiento,sentimiento
1,Compasión,perdón,por lo que se le perdona la vida a alguien en ...
2,plan,objetivos,necesario para cumplir objetivos
3,plan,organización,organizar recursos o personas
4,plan,estrategia,estrategia
...,...,...,...
4933,Obligación,imposición,imposicion
4934,Obligación,extricto,carácter extricto
4935,Obligación,normas,normas
4936,Obligación,deber,deberes


In [32]:
from sklearn.model_selection import KFold

kfold=KFold(n_splits=5)
counter = 1
for train_index, test_index in kfold.split(data_matrix):
    test = data_matrix.iloc[test_index,:]
    test.to_csv(r"../data/data_buchanan/test{}.csv".format(counter))
    counter+=1


In [43]:
buchanan_acc=[]
for i in range(5):
    test = pd.read_csv(r'../data/data_buchanan/test{}.csv'.format(i+1))
    test_result = pd.read_csv(r'../data/data_buchanan/resultados/multi.nostop.lemmas_test{}.csv'.format(i+1))

    len_test = len(test)
    counter = 0
    
    for i in range(len_test):
        counter += test['Codificación'][i] == np.random.choice(test_result['combined.lemmas'])
    buchanan_acc.append(counter/len_test)

np.mean(buchanan_acc)

0.0006076976401724442

In [6]:
# Corregir Buchanan 
# Sacar las codificaciones a partir de la descripcion en concreto
# EJ: descrip: sentimiento  =>  cod: sentimiento
from hunspell import Hunspell
import treetaggerwrapper


C:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
C:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
C:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
C:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [31]:
class BuchananProcessor:

    def __init__(self, data):
        self.data = data
        self.nlp = spacy.load('es_core_news_sm')
        self.h = Hunspell('es_CL', hunspell_data_dir='../dict')
        self.tagger = treetaggerwrapper.TreeTagger(TAGLANG='es')

    def _sentenceCheck(self, text):
        """
        Recives a string and returns the same without special caracters and 
        the correct spelling of the string in spanish
        """

        # Remove all special caracters 
        words = [t.orth_ for t in self.nlp(text) if not (t.is_punct or t.orth_==" ")]
        tokens = [t.lower() for t in words]

        c_spell = [self.h.suggest(t)[0] if (self.h.spell(t)==False and len(self.h.suggest(t))>0) else t for t in tokens]

        return " ".join(c_spell)
        
    def spellCheck(self):
        for i in range(len(self.data.iloc[:,2])):
            self.data.iloc[i,2] = self._sentenceCheck(self.data.iloc[i,2]) 

    def _lemmatize(self,text):
        """
        Recives a string and return a list with the word, pos tagging and the lemma,
        for each word
        """
        return [tag for tag in treetaggerwrapper.make_tags(self.tagger.tag_text(text), exclude_nottags=True)]
        # Mantener los tags completos porque se van a necesitar para otras funciones

    def lemmatization(self):
        self.data['lemma'] = [self._lemmatize(self.data.iloc[i,2]) for i in range(len(self.data.iloc[:,2]))]

    def stopWordRemoval(self):
        for i in range(len(self.data['lemma'])):
            self.data['lemma'][i] = [word for word in self.data['lemma'][i] if self.nlp(word[2])[0].is_stop==False]
    
    def multiWordSequence(self):
        

    def process(self):
        self.spellCheck()
        self.lemmatization()
        self.stopWordRemoval()

In [32]:
P = BuchananProcessor(data_matrix)
P.process()
P.data

,Palabra (concepto),Codificación,Descripción,lemma
0,Compasión,sentimiento,sentimiento,"[(sentimiento, NC, sentimiento)]"
1,Compasión,perdón,por lo que se le perdona la vida a alguien en ...,"[(le, PPC, él|le), (perdona, VLfin, perdonar),..."
2,plan,objetivos,necesario para cumplir objetivos,"[(necesario, ADJ, necesario), (cumplir, VLinf,..."
3,plan,organización,organizar recursos o personas,"[(organizar, VLinf, organizar), (recursos, NC,..."
4,plan,estrategia,estrategia,"[(estrategia, NC, estrategia)]"
...,...,...,...,...
4933,Obligación,imposición,imposición,"[(imposición, NC, imposición)]"
4934,Obligación,extricto,carácter extracto,"[(carácter, NC, carácter), (extracto, NC, extr..."
4935,Obligación,normas,normas,"[(normas, NC, norma)]"
4936,Obligación,deber,deberes,"[(deberes, NC, deber)]"


In [113]:
P._lemmatize("por lo que se le perdona la vana ea alguien")

['por', 'el', 'que', 'se', 'él|le', 'perdonar', 'el', 'vano', 'ea', 'alguien']

In [23]:
# for word in P.data['lemma'][1]: print(word[2])

element = P.data['lemma'][1]
for word in element:
    if P.nlp(word[2])[0].is_stop==False:
        print(word)
        

Tag(word='le', pos='PPC', lemma='él|le')
Tag(word='perdona', pos='VLfin', lemma='perdonar')
Tag(word='vida', pos='NC', lemma='vida')
Tag(word='a', pos='PREP', lemma='a')
Tag(word='alguien', pos='QU', lemma='alguien')
Tag(word='películas', pos='NC', lemma='película')


In [26]:
P.data['lemma'][1]

[Tag(word='por', pos='PREP', lemma='por'),
 Tag(word='lo', pos='ART', lemma='el'),
 Tag(word='que', pos='CQUE', lemma='que'),
 Tag(word='se', pos='SE', lemma='se'),
 Tag(word='le', pos='PPC', lemma='él|le'),
 Tag(word='perdona', pos='VLfin', lemma='perdonar'),
 Tag(word='la', pos='ART', lemma='el'),
 Tag(word='vida', pos='NC', lemma='vida'),
 Tag(word='a', pos='PREP', lemma='a'),
 Tag(word='alguien', pos='QU', lemma='alguien'),
 Tag(word='en', pos='PREP', lemma='en'),
 Tag(word='las', pos='ART', lemma='el'),
 Tag(word='películas', pos='NC', lemma='película')]